# EDA: Exploratory Data Analysis

- tools: `pandas`, `seaborn` & `matplotlib`
- concepts:
    - structure
    - granularity
    - scope 
    - temporality
    - faithfulness
    - missingness
    - single variable: shape, central tendency, variability
    - multiple variable EDA (disc./examples coming soon)

<div class="alert alert-success">
For more information on this topic, check out: (1) Jake VanderPlas' <a href="https://jakevdp.github.io/PythonDataScienceHandbook/" class="alert-link">Python Data Science Handbook</a> and (2) Berkeley's <a href="https://learningds.org/intro.html" class="alert-link">Learning Data Science</a>.
</div>

## Exploratory data analysis (EDA)

> an approach to completely and fully understand your dataset.

It requires a state of flexibility and a willingness to look for both:
- artifacts in the data we anticipate exist 
- artifacts that we don't expect / believe are there



## Setup

In [ ]:
# Import seaborn and apply its plotting styles
import seaborn as sns
sns.set(style="white", font_scale=1.5)

# import matplotlib
import matplotlib.pyplot as plt
# set plotting size parameter
plt.rcParams['figure.figsize'] = (14, 7)

# make sure pandas & numpy are imported
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

#improve resolution
#comment this line if erroring on your machine/screen
%config InlineBackend.figure_format ='retina'

## Motivating Question(s)

How well do COGS 108 students estimate the following:
- How fast does human hair grow (cm/yr)?
- If every living person stood crammed together side-by-side, how large of an area would they occupy (km²)?
- How many days would it take to walk from San Diego to New York City (assuming no stopping to fix shoes, apply sunscreen, or for sleeping, eating, or other biological needs)?

## The Data

To walk through these concepts today, we're going to use your responses from after the Data Intuition Lecture.

In [ ]:
# read data into Python
df = pd.read_csv('data/woc_wi24.csv')

In [ ]:
# take a look at the data
df

### Clicker Question #0

Are these data in the tidy data format?

- A) Yes, these data are ready to analyze
- B) Yes, but there is more work to do before analysis
- C) No, not tidy
- D) Have no idea what you're talking about

**Brainstorming**

What considerations do we have to make about these data?

### Tidy Data Rules (Review):

1. Every observation in a row
2. Every variable in a column
3. If multiple tables, column on which to merge

### Clicker Question #1

What's the granularity of this dataset? 

- A) Individual-Level
- B) Group-level
- C) Individual-level over time
- D) Group-level over time

<div class="alert alert-info">
The <b>granularity</b> of your data is what each observation in your data represents.
</div>

In [ ]:
# change column names
df.columns = ['timestamp', 'hair_growth', 'crammed', 'SAN_NYC']
df.head()

In [ ]:
# check type of each Series (column)
df.dtypes

In [ ]:
# determine rows and columns in df
df.shape

## EDA

At this point...we know:

- granularity
- temporality
- scope
- ~faithfulness

Let's check our understanding.

### Clicker Question #2

What's the granularity of this dataset? 

- A) Individual-Level
- B) Group-level
- C) Individual-level over time
- D) Group-level over time

<div class="alert alert-info">
The <b>granularity</b> of your data is what each observation in your data represents.
</div>

### Clicker Question #3

If we wanted to understand something about all COGS 108 students this quarter, could we do that with these data? 

- A) Yes
- B) No
- C) ¯\\\_(ツ)\_/¯

Note: There are currently 700 students enrolled in COGS 108.

<div class="alert alert-info">
The <b>scope</b> of your data describes how helpful these data are with respect to our data science question.
</div>

### Clicker Question #4

How trustworthy do you find these data (given what you've seen so far)?

- A) Definitely ready to use for analysis
- B) Maybe ready to use for analysis
- C) Definitely *not* ready to use for analysis
- D) ¯\\\_(ツ)\_/¯

<div class="alert alert-info">
The <b>faithfulness</b> of your data is a determination of how trustworthy the data are.
</div>

What we don't know (yet)
- missingness
- Single variables: Shape, Central Tendency, Variability
- Relationship across variables?

## Missingness

Data can be missing for all kinds of reasons. It's your job to determine if:
- values are missing at random
- values are missing due to data entry errors
- values are missing due to faulty data collection



In [ ]:
# True if row contains at least one null value
# axis argument: 0 for reducing by ‘index’, 1 for reducing by ‘columns’,
null_rows = df.isnull().any(axis=1)
df[null_rows].shape

In [ ]:
# columns with missing values
df.columns[df.isnull().any(axis=0)]

In [ ]:
# number of missing values by column
df.isnull().sum()

## Cleaning: Hair Growth

How fast does human hair grow (cm/yr)?

In [ ]:
# take a look at unique values
df["hair_growth"].unique()

In [ ]:
# standardize height column
def standardize_hair(string):
    
    # Basic string pre-processing
    string = string.lower()
    string = string.strip()    

    # take care of inclded unit cases   
    string = string.replace("cm a year", "")
    string = string.replace("cm/year", "")
    string = string.replace("cm/ye", "")
    string = string.replace("centimeters", "")
    string = string.replace("cm per year", "")
    string = string.replace("cm/yr", "")
    string = string.replace("cm/1yr", "")
    string = string.replace("cm/y", "")
    string = string.replace("cm/1 year", "")
    string = string.replace("year", "")
    string = string.replace("yr", "")
    string = string.replace("cm", "")    
    string = string.replace("/", "")
    string = string.replace(",", "")

    string = string.replace("^2", "0")
    
    string = string.strip()

    # convert to numeric
    try:
        output = float(eval(string))
    except:
        output = np.nan
    
    return output

In [ ]:
# apply function across values in hair growth columns
df["hair_growth"] = df["hair_growth"].apply(standardize_hair)
df["hair_growth"].unique()

## Cleaning: Crammed

If every living person stood crammed together side-by-side, how large of an area would they occupy (km²)?

In [ ]:
df['crammed'].unique()

In [ ]:
# standardize crammed column
def standardize_crammed(string):    
    # Basic string pre-processing
    string = string.lower()
    string = string.strip()
    
    # take care of commas
    string = string.replace(",", "")

    # take care of inclded unit cases
    string = string.replace("^2", "")
    string = string.replace("²", "")
    string = string.replace("km", "")
    
    # take care of scientific notation / word cases
    string = string.replace(" million", "000000")
    string = string.replace(" billion", "000000000")

    string = string.strip()

    # convert to numeric
    try:
        output = float(eval(string))
    except:
        output = np.nan
    
    return output

In [ ]:
# apply function across values in crammed columns
df["crammed"] = df["crammed"].apply(standardize_crammed)
df["crammed"].unique()

## Cleaning: San Diego to NYC

How many days would it take to walk from here to New York City (assuming no stopping to fix shoes, apply sunscreen, or for sleeping, eating, or other biological needs)?

In [ ]:
df['SAN_NYC'].unique()

In [ ]:
# a more general approach compared to above
def dist_helper(distance):
    if len(distance) == 1:
        return distance[0]
    if distance[1] == '*':
        return str(float(distance[0]) * float(distance[2]))
    if distance[1] == '/':
        return str(float(distance[0]) / float(distance[2]))

df["SAN_NYC"] = pd.to_numeric(df["SAN_NYC"].
                              str.lower().
                              replace({"days?":"", 
                                       "\^":"e", "^\([\w */)=?]+$":"1000",
                                       " weeks?":" * 7",
                                       " months?":" * 30", 
                                       " hours?": " / 24", 
                                       " years?": " * 365"}, regex = True).
                              str.strip().
                              str.split(' ').
                              apply(dist_helper), errors = 'coerce')


In [ ]:
# apply function across values in crammed columns
df["SAN_NYC"].unique()

#### Return to missingness

Note that after cleaning, we DO have missing values. These would be worth looking into further; being sure we understand *why* they're missing

In [ ]:
# number of missing values by column
df.isnull().sum()

In [ ]:
null_rows = df.isnull().any(axis=1)
df[null_rows]

## The Cleaned Data

In [ ]:
df

### Clicker Question #5

What could we have done to avoid these cleaning/wrangling issues?

- A) I have some ideas.
- B) I've thought but I have no idea.
- C) I'm unsure what we're supposed to be thinking about

**Brainstorming**


## Aside: Anscombe's Quartet: A Cautionary Tale

Code in this example taken from [here](https://matplotlib.org/gallery/specialty_plots/anscombe.html).

In [ ]:
x = np.array([10, 8, 13, 9, 11, 14, 6, 4, 12, 7, 5])
y1 = np.array([8.04, 6.95, 7.58, 8.81, 8.33, 9.96, 7.24, 4.26, 10.84, 4.82, 5.68])
y2 = np.array([9.14, 8.14, 8.74, 8.77, 9.26, 8.10, 6.13, 3.10, 9.13, 7.26, 4.74])
y3 = np.array([7.46, 6.77, 12.74, 7.11, 7.81, 8.84, 6.08, 5.39, 8.15, 6.42, 5.73])
x4 = np.array([8, 8, 8, 8, 8, 8, 8, 19, 8, 8, 8])
y4 = np.array([6.58, 5.76, 7.71, 8.84, 8.47, 7.04, 5.25, 12.50, 5.56, 7.91, 6.89])

In [ ]:
def fit(x):
    return 3 + 0.5 * x

xfit = np.array([np.min(x), np.max(x)])

plt.subplot(221)
plt.plot(x, y1, 'ks', xfit, fit(xfit), 'r-', lw=2)
plt.axis([2, 20, 2, 14])
plt.setp(plt.gca(), xticklabels=[], yticks=(4, 8, 12), xticks=(0, 10, 20))
plt.text(3, 12, 'I', fontsize=20)

plt.subplot(222)
plt.plot(x, y2, 'ks', xfit, fit(xfit), 'r-', lw=2)
plt.axis([2, 20, 2, 14])
plt.setp(plt.gca(), xticks=(0, 10, 20), xticklabels=[],
         yticks=(4, 8, 12), yticklabels=[], )
plt.text(3, 12, 'II', fontsize=20)

plt.subplot(223)
plt.plot(x, y3, 'ks', xfit, fit(xfit), 'r-', lw=2)
plt.axis([2, 20, 2, 14])
plt.text(3, 12, 'III', fontsize=20)
plt.setp(plt.gca(), yticks=(4, 8, 12), xticks=(0, 10, 20))

plt.subplot(224)
xfit = np.array([np.min(x4), np.max(x4)])
plt.plot(x4, y4, 'ks', xfit, fit(xfit), 'r-', lw=2)
plt.axis([2, 20, 2, 14])
plt.setp(plt.gca(), yticklabels=[], yticks=(4, 8, 12), xticks=(0, 10, 20))
plt.text(3, 12, 'IV', fontsize=20)

# verify the stats
pairs = (x, y1), (x, y2), (x, y3), (x4, y4)
for x, y in pairs:
    print('mean=%1.2f, std=%1.2f, r=%1.2f' % (np.mean(y), np.std(y),
          np.corrcoef(x, y)[0][1]))

plt.show()

### Draw the Graph (What a complete EDA is all about!)

## Shape

- range of values
- overall shape
- outliers? 

### Outliers

Outliers are values that fall outside the typical range of your dataset. These can occur for all types of reasons:

- data entry errors
- poor sampling procedures
- technical or mechanical errors
- unexpected changes in weather
- extreme values
- people giving incorrect information
- etc.

In [ ]:
sns.histplot(df['hair_growth'], bins=20);

In [ ]:
sns.histplot(df['crammed'], bins=20);

In [ ]:
sns.histplot(df['SAN_NYC'], bins=20);

<div class="alert alert-danger">
Caution: Observations should only be removed from your dataset if you have a valid reason to do so. If you remove outliers from your dataset, your report should be <b>very</b> clear that you did so.
</div>

### Clicker Question #6

Should we remove any outlier values in this dataset? 

- A) Yes
- B) No
- C) ¯\\\_(ツ)\_/¯

## Central Tendency

- mean 
- median
- mode

The Central Tendency tells you the 'typical' value for an observation in your dataset.

### Mean


$$ \bar x = \frac{\sum\limits_{i = 1}^N x_i}N  $$


- $x_i$ = ith element of the sample
- $\bar x$ = sample mean
- $N$ = sample size


In [ ]:
df['hair_growth'].mean()

In [ ]:
# check mean for each column
df.mean()

### Median

In [ ]:
# check median for each column
df.median()

### Median vs. Mean

When the median and mean are not similar to one another...what's the best approach?

In [ ]:
# median and mean for same series
print( 'median: ', df['hair_growth'].median())
print( 'mean: ', df['hair_growth'].mean())

In [ ]:
# take a look at it all together
ax = sns.histplot(df['hair_growth'], bins=5);
ax.axvline(df['hair_growth'].mean(), color='darkred', linestyle='--', label='mean');
ax.axvline(df['hair_growth'].median(), color='#2e2e2e', linestyle='-', label='median')
ax.legend();

### Clicker Question #7

Which of the following is the best way to measure the central tendency of `hair_growth` in these data?

- A) mean
- B) median
- C) mode

In [ ]:
# increase the number of bins here
ax = sns.histplot(df['hair_growth'], bins=100);
ax.axvline(df['hair_growth'].mean(), color='darkred', linestyle='--', label='mean');
ax.axvline(df['hair_growth'].median(), color='#2e2e2e', linestyle='--', label='median')
ax.legend();

#### How did y'all do?

How well do COGS 108 students estimate the following:
- How fast does human hair grow (cm/yr)?
- If every living person stood crammed together side-by-side, how large of an area would they occupy (km²)?
- How many days would it take to walk from San Diego to New York City (assuming no stopping to fix shoes, apply sunscreen, or for sleeping, eating, or other biological needs)?

In [ ]:
# compare to actual value: 15 cm/year (~6 in)
df["hair_growth"].median()

In [ ]:
# compare to actual value: 1,000-10,000 km^2)
df['crammed'].median()

In [ ]:
# compare to actual value: 38 days)
df['SAN_NYC'].median()

Calculating the mean and median of your sample is helpful when dealing with **quantitative variables**.

When working with **categorical variables**, knowing the mode is helpful.

### Mode

When working with categorical data, the mode is the most common value in the dataset.

### Aside: Why Central Tendency Doesn't Tell the Whole Story

In [ ]:
# generate two different normal distributions
dist_1 = np.random.normal(5, 2, 1000)
dist_2 = np.random.normal(5, 10, 1000)

In [ ]:
# plot distributions side by side
sns.kdeplot(dist_1, label="mean: 5, sd: 2")
sns.kdeplot(dist_2, label="mean: 5, sd: 10")
plt.legend();

## Variability

- Range
- IQR 
- Variance & Standard Deviation

### Range

The highest value minus the lowest value.

In [ ]:
# determine the 25th and 75th percentiles
min_val = df['hair_growth'].min()
max_val = df['hair_growth'].max()
range_vals =  max_val - min_val
print(max_val, '-' , min_val,' = ',  range_vals)

### IQR (Interquartile Range)

75th percentile - 25th percentile

In [ ]:
# determine the 25th and 75th percentiles
lower, upper = np.percentile(df['hair_growth'], [25, 75])
lower, upper

In [ ]:
# calculate IQR
iqr = upper - lower
iqr

In [ ]:
# visualizing IQR
sns.boxplot(x='hair_growth', data=df);

### Variance & Standard Deviation

- variance
    - measures how close the values in the distribution are to the middle of the distribution
    - average squared difference of the scores from  the mean
- standard deviation
    - square root of the variance

#### Variance

$$ s^2 = \frac {\sum\limits_{i = 1}^N {\left( {x_i - \bar x} \right)^2 }} {(N-1)} $$

- $s^2$ = sample variance
- $x_i$ = ith element of the sample
- $\bar x$ = mean of the sample
- $N$ = sample size


In [ ]:
# calculate variance using pandas
var = df['hair_growth'].var()
var

#### Standard Deviation
square root of the variance

$$ s = \sqrt {\frac {\sum\limits_{i = 1}^N {\left( {x_i - \bar x} \right)^2 }} {(N-1)}} $$

In [ ]:
np.sqrt(var)

In [ ]:
# calculate variance using pandas
sd = df['hair_growth'].std()
sd 

## Descriptive Tables

Helpful for understanding your data...but not the best explanatory visualization.

In [ ]:
df.describe()

### Clicker Question #8

What would be the next step in this analysis to answer our question of interest?

- A) I have some ideas on next steps.
- B) I think the analysis is complete.
- C) ¯\\\_(ツ)\_/¯

### Clicker Question #9

Final thoughts: Should you ensure that all plots you make during the EDA process adhere to data visualization best practices?
    
- A) Yes.
- B) No.
- C) ¯\\\_(ツ)\_/¯